In [1]:
import requests
import urllib
import math
import time
import random

import pandas as pd
import sqlite3

In [2]:
my_headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Host': 'music.163.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
}

In [3]:
def getJSON(url, headers):
    """
    @ param url:
    @ param headers:
    @ return json:
    """
    res = requests.get(url, headers=headers)
    res.raise_for_status()  
    res.encoding = 'utf-8'  
    json = res.json()
    return json

In [4]:
def countPages(total, limit):
    """
    @ param total:
    @ return page:
    """
    page = math.ceil(total/limit) 
    return page

In [5]:
def getSongInfo(song_list):
    """
    
    """
    song_info_list = []
    
    for song in song_list:
        song_info = []
    
        song_info.append(song['id'])
        song_info.append(song['name'])
    
        artist_str = ''
        artists = song['artists']
        for artist in artists:
            artist_str += artist['name'] + '/'
        song_info.append(artist_str)
    
        song_info.append(song['album']['name'])
        song_info.append(song['duration'])
        
        song_info_list.append(song_info)
        
    return song_info_list

In [6]:
def getSongList(key, limit=30):
    """
    @ param key:
    @ return result:
    """
    total_list = []
    key = urllib.parse.quote(key)
    url = 'http://music.163.com/api/search/get/web?csrf_token=&hlpretag=&hlposttag=&s=' + key +  '&type=1&offset=0&total=true&limit='
    
    first_page = getJSON(url, my_headers)
    song_count = first_page['result']['songCount']
    page_num = countPages(song_count, limit)
    
    for n in range(page_num):
        url = 'http://music.163.com/api/search/get/web?csrf_token=&hlpretag=&hlposttag=&s=' + key +  '&type=1&offset=' + str(n) + '&total=true&limit=' + str(limit)
        tmp = getJSON(url, my_headers)
        song_list = getSongInfo(tmp['result']['songs'])
        total_list += song_list
        
        print('第 {0}/{1} 页爬取完成'.format(n+1, page_num))
        time.sleep(random.randint(2, 8)) 
        
    df = pd.DataFrame(data = total_list)
    return df

In [7]:
def getSongComment():
    """
    
    """

In [8]:
def getSongDetail(id):
    """
    @ param id:
    @ return result:
    """

In [ ]:
getJSON(url, my_headers)